# Chapter 7 - 함수 데커레이터와 클로저

함수 데커레이터: 소스 코드에 있는 함수를 '표시'해서 함수의 작동을 개선할 수 있게 해준다.<br>
강력한 기능이지만 데커레이터를 자유자재로 사용하려면 클로저를 알아야 한다.

## 데커레이터 기본 지식

데커레이터는 다른 함수를 인수로 받는 콜러블이다.

```python
@decorate
def target():
    print('running target()')
```

위 코드는 아래 코드와 동일하게 동작한다.

```python
def target():
    print('running target()')

target = decorate(target)
```

## 파이썬이 데커레이터를 실행하는 시점

데커레이터된 함수가 정의된 직후에 실행된다. 즉, **임포트 타임**에 실행된다.

In [1]:
def register(func):
    print(f'running register({func})')
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

running register(<function f1 at 0x7fd25cce29e0>)
running register(<function f2 at 0x7fd25cce2cb0>)
running main()
running f1()
running f2()
running f3()


## 변수 범위 규칙

파이썬은 변수가 선언되어 있기를 요구하지 않지만, 함수 본체 안에서 할당된 변수는 지역 변수로 판단한다.

In [2]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

In [3]:
b = 6
def f2(a):
    print(a)
    print(b)
    
f2(3)

3
6


In [4]:
b = 9
def f3(a):
    print(a)
    print(b)
    b = 12
    
f3(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [5]:
def f4(a):
    print(a)
    print(b)
    
b = 15
f4(3)

3
15


## 클로저

클로저는 함수 본체에서 정의하지 않고 참조하는 비전역 변수를 포함한 확장 범위를 가진 함수다.<br>
함수가 익명 함수인지 여부는 중요하지 않다. 함수 본체 외부에 정의된 비전역 변수에 접근할 수 있다는 것이 중요하다.

In [6]:
class Averager():
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)
    
avg = Averager()
print(avg(10))
print(avg(20))
print(avg(30))

10.0
15.0
20.0


In [2]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

avg1 = make_averager()
print(avg1(10))
print(avg1(20))
print(avg1(30))

10.0
15.0
20.0


In [4]:
avg2 = make_averager()
print(avg2(10))  # 새로운 series가 생성되는지 확인
print(avg2(20))
print(avg2(30))

print(avg2.__code__.co_varnames)
print(avg2.__code__.co_freevars)
print(avg2.__closure__)
print(avg2.__closure__[0].cell_contents)

10.0
15.0
20.0
('new_value', 'total')
('series',)
(<cell at 0x7f7c58191910: list object at 0x7f7c58220550>,)
[10, 20, 30]


![](https://i.ibb.co/y6NZZGL/asdf.png)

## nonlocal 선언

In [5]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1             # count에 할당이 일어나서 지역변수로 취급 -> 오류 발생
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
print(avg(10))

UnboundLocalError: local variable 'count' referenced before assignment

![](https://i.ibb.co/vmx6wBb/fdsa.png)

In [7]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total  # nonlocal로 해결
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
print(avg(10))
print(avg(20))
print(avg(30))

10.0
15.0
20.0


## 간단한 데커레이터 구현하기

In [11]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print(f'6! = {factorial(6)}')

**************************************** Calling snooze(.123)
[0.12316813s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000059s] factorial(1) -> 1
[0.00003148s] factorial(2) -> 2
[0.00006023s] factorial(3) -> 6
[0.00008014s] factorial(4) -> 24
[0.00010063s] factorial(5) -> 120
[0.00012111s] factorial(6) -> 720
6! = 720


### functools.lru_cache()를 이용한 메모이제이션

functools.lru_cahce() 데커레이터는 실제로 쓸모가 많은 데커레이터로써, 메모이제이션을 구현한다.<br>
lru_cache()의 전체 시그니처는 다음과 같다.
```python
functools.lru_cache(maxsize=128, typed=False)
```
maxsize: 얼마나 많은 호출을 저장할지 결정한다. 최적의 성능을 위해 2의 제곱꼴이어야 한다.<br>
typed: True로 설정되는 경우 인수의 자료형이 다르면 결과를 따로 저장한다.<br>
<br>
다음은 필자가 직접 백준온라인저지에서 functools.lru_cache()의 힘을 확인하기 위해 실험한 사진이다.<br>
실험은 [1003번-피보나치 함수](https://www.acmicpc.net/problem/1003)로 진행하였다.<br>
입력으로 들어오는 N이 최대 40이며, 이를 직접 재귀호출을 통해 구할 경우 3억 3116만 281번이 이루어진다.<br>

![](https://i.ibb.co/fQkpVzD/image.png)<br>
재귀를 순수하게 모두 수행한 코드이다. 시간초과를 받았다.<br>
<br>
![](https://i.ibb.co/RpkSmtk/1.png)<br>
functools.lru_cache() 데커레이터를 통해 메모이제이션을 적용한 코드이다.<br>
위의 코드와 다른 점은 상단에 2줄의 코드만 추가한 것 뿐이다. 96ms로 통과를 받았다.